In [1]:
import numpy as np
import cv2

# Load map in grayscale
map = cv2.imread("map.png", 0)
HEIGHT, WIDTH = map.shape

# Initialize robot's position and orientation
rx, ry, rtheta = WIDTH / 4, HEIGHT / 4, 0

# Display the map to ensure it's loaded correctly
cv2.imshow('Map', map)
cv2.waitKey(0)
cv2.destroyAllWindows()




In [2]:
STEP = 5
TURN = np.radians(45)
NUM_PARTICLES = 5000
SIGMA_STEP = 0.5
SIGMA_TURN = np.radians(5)
SIGMA_SENSOR = 2
SIGMA_POS = 2
SIGMA_TURN_NOISE = np.radians(10)



In [3]:
def get_input():
    fwd = 0
    turn = 0
    halt = False
    k = cv2.waitKey(0)
    if k == ord('w'):  # Move forward
        fwd = STEP
    elif k == ord('s'):  # Move backward
        fwd = -STEP
    elif k == ord('a'):  # Turn left
        turn = -TURN
    elif k == ord('d'):  # Turn right
        turn = TURN
    elif k == 27:  # ESC key to exit
        halt = True
    return fwd, turn, halt



In [4]:
def move_robot(rx, ry, rtheta, fwd, turn):
    fwd_noisy = np.random.normal(fwd, SIGMA_STEP)
    rx += fwd_noisy * np.cos(rtheta)
    ry += fwd_noisy * np.sin(rtheta)
    
    turn_noisy = np.random.normal(turn, SIGMA_TURN)
    rtheta += turn_noisy
    
    return rx, ry, rtheta


In [5]:
def init_particles():
    particles = np.random.rand(NUM_PARTICLES, 3)
    particles[:, 0] *= WIDTH
    particles[:, 1] *= HEIGHT
    particles[:, 2] *= np.radians(360)
    return particles



In [6]:
def move_particles(particles, fwd, turn):
    velocity_factor = 2.0  # Ensure this factor is appropriately scaled
    particles[:, 0] += velocity_factor * fwd * np.cos(particles[:, 2])
    particles[:, 1] += velocity_factor * fwd * np.sin(particles[:, 2])
    particles[:, 2] += turn
    
    particles[:, 0] = np.clip(particles[:, 0], 0.0, WIDTH - 1)
    particles[:, 1] = np.clip(particles[:, 1], 0.0, HEIGHT - 1)
    return particles



In [7]:
def sense(x, y, noisy=False):
    x = int(x)
    y = int(y)
    if noisy:
        return np.random.normal(map[y, x], SIGMA_SENSOR)
    return map[y, x]

def compute_weights(particles, robot_sensor):
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        elevation = sense(particles[i, 0], particles[i, 1], noisy=False)
        errors[i] = abs(robot_sensor - elevation)
    
    weights = np.max(errors) - errors
    weights[
        (particles[:, 0] == 0) |
        (particles[:, 0] == WIDTH - 1) |
        (particles[:, 1] == 0) |
        (particles[:, 1] == HEIGHT - 1)
    ] = 0.0

    # Raise weights to the fourth power for more pronounced effects
    weights = weights ** 4
    return weights



In [8]:
def resample(particles, weights):
    # Normalize weights to get valid PDF
    probabilities = weights / np.sum(weights)

    # Resample
    new_index = np.random.choice(NUM_PARTICLES, size=NUM_PARTICLES, p=probabilities)
    particles = particles[new_index, :]
    return particles


In [9]:
def add_noise(particles):
    noise = np.concatenate((
        np.random.normal(0, SIGMA_POS, (NUM_PARTICLES, 1)),
        np.random.normal(0, SIGMA_POS, (NUM_PARTICLES, 1)),
        np.random.normal(0, SIGMA_TURN_NOISE, (NUM_PARTICLES, 1)),
    ), axis=1)
    particles += noise
    return particles


In [10]:
def display(map, rx, ry, particles):
    lmap = cv2.cvtColor(map, cv2.COLOR_GRAY2BGR)
    
    if len(particles) > 0:
        for i in range(NUM_PARTICLES):
            cv2.circle(lmap, (int(particles[i, 0]), int(particles[i, 1])), 1, (255, 0, 0), 1)
    
    cv2.circle(lmap, (int(rx), int(ry)), 5, (0, 255, 0), 10)
    
    if len(particles) > 0:
        px = np.mean(particles[:, 0])
        py = np.mean(particles[:, 1])
        cv2.circle(lmap, (int(px), int(py)), 5, (0, 0, 255), 5)
    
    cv2.imshow('Map', lmap)


particles = init_particles()
while True:
    display(map, rx, ry, particles)
    fwd, turn, halt = get_input()
    if halt:
        break
    
    rx, ry, rtheta = move_robot(rx, ry, rtheta, fwd, turn)
    particles = move_particles(particles, fwd, turn)
    
    if fwd != 0:
        robot_sensor = sense(rx, ry, noisy=True)
        weights = compute_weights(particles, robot_sensor)
        particles = resample(particles, weights)
        particles = add_noise(particles)
    
cv2.destroyAllWindows()
